In [7]:
import requests
import lxml.html as lh
import pandas as pd
import numpy as np
import PyPDF2
import tabula

### Methods


In [8]:
def get_files_list(url):
    #Create a handle, page, to handle the contents of the website
    page = requests.get(url)
    #Store the contents of the website under doc
    doc = lh.fromstring(page.content)
    #look for links in the website
    tree = lh.fromstring(page.text)
    links = []
    for link_element in tree.xpath('//td//a')[1:]:
        links.append(link_element.get('href'))
    return links

def save_list(li):
    with open('FilesList.txt', 'w') as filehandle:
        for listitem in li:
            filehandle.write('%s\n' % listitem)
        print('List saved as FilesList.txt')

def load_list(): 
    lines = []
    with open('FilesList.txt', 'r') as filehandle:        
        lines = [line[:-1] for line in filehandle]
    return lines

def download_file(f):
    db_url = "https://www.iedcr.gov.bd/website/images/files/nCoV/"
    url = db_url + f
    response = requests.get(url)
    file_name = url.split('/')[-1:][0]
    with open(file_name, 'wb') as f:
        f.write(response.content)
        print(file_name +' saved!')

def get_new_files(url):
    links = get_files_list(url)
    #check for updates
    old = load_list()
    updates = []
    for item in links :    
        if item not in old:
            updates.append(item)
    #download new files in site
    for item in updates:
        download_file(item)
    #save current list
    save_list(links)
    print("new files from website saved")

def get_latest_dl(): #get file name from downloads list
    with open('FilesList.txt', 'r') as filehandle:        
        lines = [line for line in filehandle]
    return lines[0].split()[0]


def pdf_to_csv(file = get_latest_dl()): #input pdf name or by default uses latest file
    df = tabula.read_pdf(file, pages = [1, 2])#does not read first table
    final = df[1]
    final.columns = ['Location' , 'Total']
    for idx , dfa in enumerate(df[2:]):
        x = dfa[:].values
        x = np.append(x, [final.columns[0] , final.columns[1]])
        dataset = pd.DataFrame({final.columns[0]: x[0:-2:2], final.columns[1]: x[1:-2:2]})
        final = pd.concat([final , dataset])

    final = final.reset_index(drop = True)

    final.to_csv('corona.csv' , index = False)
    print('saved as corona.csv')

In [9]:
url = 'https://www.iedcr.gov.bd/website/images/files/nCoV/?C=M;O=D'
get_new_files(url)

List saved as FilesList.txt
new files from website saved


In [10]:
pdf_to_csv()

Got stderr: Jun 13, 2020 11:46:58 AM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font ABCDEE+Calibri are not implemented in PDFBox and will be ignored
Jun 13, 2020 11:47:00 AM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font ABCDEE+Calibri are not implemented in PDFBox and will be ignored
Jun 13, 2020 11:47:02 AM org.apache.pdfbox.pdmodel.font.PDCIDFontType2 <init>
INFO: OpenType Layout tables used in font ABCDEE+Calibri are not implemented in PDFBox and will be ignored



saved as corona.csv


In [4]:
# pdf = open(get_latest_dl() ,'rb')
# pdfReader = PyPDF2.PdfFileReader(pdf)
# extracted_text = ""

# # number of pages in pdf
# print(pdfReader.numPages)
# # a page object
# for page in range(1,2):
#     pageObj = pdfReader.getPage(page)
#     # extracting text from page.
#     extracted_text+= pageObj.extractText()
# aux = extracted_text.split()[12:]
# aux

3
